In [1]:
import argparse
import copy
import random
import tqdm
import json

import gym
import torch

import memory
from memory import Transition
import networks
from utils import np_to_unsq_tensor, squeeze_np
from experiment_utils import Plot

In [9]:
# env_name = 'Pendulum-v1'
env_name = 'CartPole-v0'
env = gym.make(env_name, render_mode="human")
state_dim = env.observation_space.shape[0]

/Users/alpayozkan/opt/anaconda3/envs/distrl_proj/lib/python3.11/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [10]:
act_dim = env.action_space

In [11]:
env.action_space.shape

()

In [7]:
from collections import OrderedDict


In [8]:
n_timesteps = 50000.0
hyperparams = \
OrderedDict([('batch_size', 64),
             ('buffer_size', 100000),
             ('exploration_final_eps', 0.04),
             ('exploration_fraction', 0.16),
             ('gamma', 0.99),
             ('gradient_steps', 128),
             ('learning_rate', 0.0023),
             ('learning_starts', 1000),
            #  ('n_timesteps', 50000.0),
             ('policy', 'MlpPolicy'),
             ('policy_kwargs', 'dict(net_arch=[256, 256], n_quantiles=10)'),
             ('target_update_interval', 10),
             ('train_freq', 256),
            #  ('normalize', False)
             ])

In [9]:
hyperparams['n_timesteps']

KeyError: 'n_timesteps'

In [10]:
n_timesteps = 50000.0
hyperparams = \
OrderedDict([('batch_size', 64),
             ('buffer_size', 100000),
             ('exploration_final_eps', 0.04),
             ('exploration_fraction', 0.16),
             ('gamma', 0.99),
             ('gradient_steps', 128),
             ('learning_rate', 0.0023),
             ('learning_starts', 1000),
             ('n_timesteps', 50000.0),
             ('policy', 'MlpPolicy'),
             ('policy_kwargs', 'dict(net_arch=[256, 256], n_quantiles=10)'),
             ('target_update_interval', 10),
             ('train_freq', 256),
             ('normalize', False)
             ])

In [11]:
hyperparams

OrderedDict([('batch_size', 64),
             ('buffer_size', 100000),
             ('exploration_final_eps', 0.04),
             ('exploration_fraction', 0.16),
             ('gamma', 0.99),
             ('gradient_steps', 128),
             ('learning_rate', 0.0023),
             ('learning_starts', 1000),
             ('n_timesteps', 50000.0),
             ('policy', 'MlpPolicy'),
             ('policy_kwargs', 'dict(net_arch=[256, 256], n_quantiles=10)'),
             ('target_update_interval', 10),
             ('train_freq', 256),
             ('normalize', False)])

In [13]:
import gymnasium as gym

from sb3_contrib import QRDQN

env = gym.make("CartPole-v1", render_mode="human")

policy_kwargs = dict(n_quantiles=50)
model = QRDQN(
              hyperparams['policy'], 
              env,
              batch_size=hyperparams['batch_size'],
              buffer_size=hyperparams['buffer_size'],
              exploration_final_eps=hyperparams['exploration_final_eps'],
              exploration_fraction=hyperparams['exploration_fraction'],
              gamma=hyperparams['gamma'],
              gradient_steps=hyperparams['gradient_steps'],
              learning_rate=hyperparams['learning_rate'],
              learning_starts=hyperparams['learning_starts'],
              
              policy_kwargs=eval(hyperparams['policy_kwargs']),
              target_update_interval=hyperparams['target_update_interval'],
              train_freq=hyperparams['train_freq'],
              verbose=1
              )




Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [35]:
import torch

test = torch.cat([torch.tensor([i]) for i in range(4)])
print(test, type(test))

tensor([0, 1, 2, 3]) <class 'torch.Tensor'>


In [43]:
sensitivities = torch.arange(0, 10, 1)
avg_sensitivities = torch.tensor([torch.true_divide(sensitivities[i] + sensitivities[i+1], 2) for i in range(sensitivities.shape[0]-1)])
print(sensitivities.shape)
print(avg_sensitivities)

torch.Size([10])
tensor([0.5000, 1.5000, 2.5000, 3.5000, 4.5000, 5.5000, 6.5000, 7.5000, 8.5000])


In [48]:
z = torch.arange(10, 0, -1).unsqueeze(0).unsqueeze(0).repeat(32, 2, 1)

z_new = torch.sum((z[:, :, :-1] - z[:, :, 1:]) * avg_sensitivities, dim=2)

print(z_new.shape, z_new)

torch.Size([32, 2]) tensor([[40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000],
        [40.5000, 40.5000]])


In [ ]:
model.learn(total_timesteps=hyperparams['n_timesteps'], 
            log_interval=4)
model.save("qrdqn_cartpole")

del model # remove to demonstrate saving and loading

model = QRDQN.load("qrdqn_cartpole")

obs, _ = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    env.render()
    if terminated or truncated:
      obs, _ = env.reset()

In [42]:
eval(hyperparams['policy_kwargs'])

{'net_arch': [256, 256], 'n_quantiles': 10}

In [53]:
t1 = torch.randn(4)
t2 = torch.tensor([2])

t_cat = torch.cat((t1, t2))
print(t_cat)

tensor([ 1.8049, -0.3140, -0.1012,  0.4776,  2.0000])


In [57]:
import numpy as np
n_gammas = 1


# Following theoretical results we want gammas to be equidistant in the 1/(log(gamma)) space
start = 1 / np.log(0.99)  
end = 1 / np.log(0.99)   
gammas = torch.exp(torch.true_divide(1, torch.linspace(start, end, n_gammas)))

print(gammas)

tensor([0.9900])


In [26]:
hyperparams

OrderedDict([('batch_size', 64),
             ('buffer_size', 100000),
             ('exploration_final_eps', 0.04),
             ('exploration_fraction', 0.16),
             ('gamma', 0.99),
             ('gradient_steps', 128),
             ('learning_rate', 0.0023),
             ('learning_starts', 1000),
             ('policy', 'MlpPolicy'),
             ('policy_kwargs', 'dict(net_arch=[256, 256], n_quantiles=10)'),
             ('target_update_interval', 10),
             ('train_freq', 256)])

In [18]:
import gymnasium as gym

from sb3_contrib import QRDQN

env = gym.make("CartPole-v1", render_mode="human")

policy_kwargs = dict(n_quantiles=50)
model = QRDQN(
   env,
   batch_size=64,
   buffer_size=100000,
   exploration_final_eps=0.04,
   exploration_fraction=0.16,
   gamma=0.99,
   gradient_steps=128,
   learning_rate=0.0023,
   learning_starts=1000,
   policy='MlpPolicy',
   verbose=1)

model.learn(total_timesteps=10_000, log_interval=4)
model.save("qrdqn_cartpole")

del model # remove to demonstrate saving and loading

model = QRDQN.load("qrdqn_cartpole")

obs, _ = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    env.render()
    if terminated or truncated:
      obs, _ = env.reset()

TypeError: QRDQN.__init__() got multiple values for argument 'policy'

In [ ]:
model.learn(total_timesteps=10_000, log_interval=4)
model.save("qrdqn_cartpole")

del model # remove to demonstrate saving and loading

model = QRDQN.load("qrdqn_cartpole")

obs, _ = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    env.render()
    if terminated or truncated:
      obs, _ = env.reset()